In [ ]:
# Import libraries for visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, confusion_matrix, precision_recall_curve,
    roc_curve, auc, precision_score, recall_score, f1_score,
    accuracy_score, roc_auc_score, average_precision_score
)

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Load results_df from previous notebook or a saved file
results_df = pd.read_csv('../data/model_results.csv')  # or pickle if you saved it


In [ ]:
# Model comparison visualization
if len(results_df) > 0:
    # Performance metrics heatmap
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # 1. Performance heatmap
    metrics_cols = ['Precision', 'Recall', 'F1', 'ROC_AUC', 'PR_AUC']
    heatmap_data = results_df.set_index('Model')[metrics_cols]

    sns.heatmap(heatmap_data.T, annot=True, fmt='.3f', cmap='RdYlGn',
                ax=axes[0,0], cbar_kws={'label': 'Score'})
    axes[0,0].set_title('Model Performance Heatmap', fontweight='bold')

    # 2. F1 Score comparison
    f1_scores = results_df.sort_values('F1', ascending=True)
    axes[0,1].barh(f1_scores['Model'], f1_scores['F1'], color='skyblue', alpha=0.7)
    axes[0,1].set_xlabel('F1 Score')
    axes[0,1].set_title('F1 Score Comparison', fontweight='bold')
    axes[0,1].grid(True, alpha=0.3)

    # 3. Precision vs Recall
    axes[1,0].scatter(results_df['Precision'], results_df['Recall'],
                     c=results_df['F1'], cmap='viridis', s=100, alpha=0.7)
    for i, model in enumerate(results_df['Model']):
        axes[1,0].annotate(model,
                          (results_df.iloc[i]['Precision'], results_df.iloc[i]['Recall']),
                          xytext=(5, 5), textcoords='offset points', fontsize=9)
    axes[1,0].set_xlabel('Precision')
    axes[1,0].set_ylabel('Recall')
    axes[1,0].set_title('Precision vs Recall Trade-off', fontweight='bold')
    axes[1,0].grid(True, alpha=0.3)

    # 4. ROC vs PR AUC
    axes[1,1].scatter(results_df['ROC_AUC'], results_df['PR_AUC'],
                     c=results_df['F1'], cmap='plasma', s=100, alpha=0.7)
    for i, model in enumerate(results_df['Model']):
        axes[1,1].annotate(model,
                          (results_df.iloc[i]['ROC_AUC'], results_df.iloc[i]['PR_AUC']),
                          xytext=(5, 5), textcoords='offset points', fontsize=9)
    axes[1,1].set_xlabel('ROC AUC')
    axes[1,1].set_ylabel('PR AUC')
    axes[1,1].set_title('ROC AUC vs PR AUC', fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Best model summary
    best_f1_model = results_df.loc[results_df['F1'].idxmax()]
    print("\n=== BEST MODEL BY F1-SCORE ===")
    print(f"Model: {best_f1_model['Model']}")
    print(f"F1-Score: {best_f1_model['F1']:.4f}")
    print(f"Precision: {best_f1_model['Precision']:.4f}")
    print(f"Recall: {best_f1_model['Recall']:.4f}")
    print(f"PR-AUC: {best_f1_model['PR_AUC']:.4f}")


In [ ]:
import pandas as pd
import joblib
import os

X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv').squeeze()
y_test = pd.read_csv('../data/y_test.csv').squeeze()
# Load results dataframe
results_df = pd.read_csv('../data/model_results.csv')

# Load test data
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv').squeeze()  # make it Series

# Load trained models
model_names = ['Logistic_Regression', 'Random_Forest', 'Decision_Tree', 'SVM','XGBoost']
trained_models = {name: joblib.load(f'../models/{name}.pkl') for name in model_names}

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

if len(results_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # ROC Curves
    for name, pipeline in trained_models.items():
        try:
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            roc_auc = auc(fpr, tpr)
            axes[0].plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')
        except Exception as e:
            print(f"⚠️ Could not plot ROC for {name}: {e}")

    axes[0].plot([0, 1], [0, 1], 'k--', label='Random')
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('ROC Curves Comparison', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Precision-Recall Curves
    for name, pipeline in trained_models.items():
        try:
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
            pr_auc = auc(recall, precision)
            axes[1].plot(recall, precision, label=f'{name} (AUC = {pr_auc:.3f})')
        except Exception as e:
            print(f"⚠️ Could not plot PR for {name}: {e}")

    fraud_rate = y_test.mean()
    axes[1].axhline(y=fraud_rate, color='k', linestyle='--', label=f'Random (={fraud_rate:.3f})')

    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    axes[1].set_title('Precision-Recall Curves Comparison', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("📊 Curve Analysis:")
    print("• ROC curves show model discrimination ability")
    print("• PR curves more informative for imbalanced data")
    print("• Higher AUC indicates better performance")


In [ ]:

# Confusion Matrices for all models
if len(results_df) > 0:
    n_models = len(trained_models)
    fig, axes = plt.subplots(1, n_models, figsize=(4*n_models, 4))

    if n_models == 1:
        axes = [axes]

    for idx, (name, pipeline) in enumerate(trained_models.items()):
        y_pred = pipeline.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)

        # Create confusion matrix heatmap
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
        axes[idx].set_title(f'{name}\nConfusion Matrix', fontweight='bold')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
        axes[idx].set_xticklabels(['Non-Fraud', 'Fraud'])
        axes[idx].set_yticklabels(['Non-Fraud', 'Fraud'])

    plt.tight_layout()
    plt.show()

    # Detailed confusion matrix analysis
    print("\n📊 Confusion Matrix Analysis:")
    print("=" * 50)
    for name, pipeline in trained_models.items():
        y_pred = pipeline.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)

        tn, fp, fn, tp = cm.ravel()
        print(f"\n{name}:")
        print(f"  True Negatives:  {tn:4d} (Correct non-fraud)")
        print(f"  False Positives: {fp:4d} (Incorrect fraud alerts)")
        print(f"  False Negatives: {fn:4d} (Missed fraud cases)")
        print(f"  True Positives:  {tp:4d} (Correct fraud detection)")
        print(f"  → Fraud Detection Rate: {tp/(tp+fn):.1%}")
        print(f"  → False Alert Rate: {fp/(fp+tn):.1%}")

In [ ]:
# Feature Importance Analysis
if len(results_df) > 0:
    print("=== Feature Importance Analysis ===")

    # Get the best performing model
    best_model_name = results_df.loc[results_df['F1'].idxmax()]['Model']
    best_pipeline = trained_models[best_model_name]

    X = pd.read_csv('../data/X.csv')
    y = pd.read_csv('../data/y.csv')

    # Extract feature importance based on model type
    feature_names = X.columns

    try:
        if hasattr(best_pipeline.named_steps['classifier'], 'feature_importances_'):
            # Tree-based models (Random Forest, Decision Tree, XGBoost)
            importances = best_pipeline.named_steps['classifier'].feature_importances_
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values('Importance', ascending=False)

        elif hasattr(best_pipeline.named_steps['classifier'], 'coef_'):
            # Linear models (Logistic Regression, SVM)
            coefficients = best_pipeline.named_steps['classifier'].coef_[0]
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': np.abs(coefficients)  # Use absolute values for importance
            }).sort_values('Importance', ascending=False)

        else:
            print(f"⚠️ Feature importance not available for {best_model_name}")
            feature_importance = None

        if feature_importance is not None:
            # Display top 15 features
            print(f"\n🔍 Top 15 Features for {best_model_name}:")
            print(feature_importance.head(15))

            # Visualization
            plt.figure(figsize=(12, 8))
            top_features = feature_importance.head(15)

            plt.barh(range(len(top_features)), top_features['Importance'])
            plt.yticks(range(len(top_features)), top_features['Feature'])
            plt.xlabel('Feature Importance')
            plt.title(f'Top 15 Feature Importance - {best_model_name}', fontweight='bold')
            plt.gca().invert_yaxis()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

            # Feature importance insights
            print(f"\n💡 Feature Importance Insights:")
            print(f"• Most important feature: {feature_importance.iloc[0]['Feature']}")
            print(f"• Top 5 features account for {feature_importance.head(5)['Importance'].sum()/feature_importance['Importance'].sum():.1%} of total importance")
            print(f"• Financial features in top 10: {sum(1 for f in feature_importance.head(10)['Feature'] if 'Amt' in f or 'sum' in f or 'mean' in f)}")

    except Exception as e:
        print(f"❌ Error extracting feature importance: {e}")

In [ ]:
# Cross-Validation Analysis
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
import numpy as np
print("=== Cross-Validation Analysis ===")

if len(results_df) > 0:
    from sklearn.model_selection import cross_validate

    # Perform cross-validation for the best model
    best_model_name = results_df.loc[results_df['F1'].idxmax()]['Model']
    best_pipeline = trained_models[best_model_name]

    # Define scoring metrics
    scoring = ['precision', 'recall', 'f1', 'roc_auc']

    print(f"\nPerforming 5-fold cross-validation for {best_model_name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    try:
        cv_results = cross_validate(
            best_pipeline, X_train, y_train,
            cv=cv, scoring=scoring,
            return_train_score=True, n_jobs=-1
        )

        # Calculate statistics
        cv_stats = {}
        for metric in scoring:
            test_scores = cv_results[f'test_{metric}']
            train_scores = cv_results[f'train_{metric}']

            cv_stats[metric] = {
                'test_mean': np.mean(test_scores),
                'test_std': np.std(test_scores),
                'train_mean': np.mean(train_scores),
                'train_std': np.std(train_scores),
                'overfit': np.mean(train_scores) - np.mean(test_scores)
            }

        # Display results
        print(f"\n📊 Cross-Validation Results for {best_model_name}:")
        print("=" * 65)
        print(f"{'Metric':<12} {'Test Mean':<10} {'Test Std':<10} {'Train Mean':<11} {'Overfitting':<12}")
        print("-" * 65)

        for metric, stats in cv_stats.items():
            print(
                f"{metric.upper():<12} {stats['test_mean']:.3f}      {stats['test_std']:.3f}      {stats['train_mean']:.3f}       {stats['overfit']:.3f}")

        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.ravel()

        for idx, metric in enumerate(scoring):
            test_scores = cv_results[f'test_{metric}']
            train_scores = cv_results[f'train_{metric}']

            x_pos = range(1, len(test_scores) + 1)
            axes[idx].plot(x_pos, test_scores, 'o-', label='Test', color='blue', alpha=0.7)
            axes[idx].plot(x_pos, train_scores, 's-', label='Train', color='red', alpha=0.7)
            axes[idx].axhline(y=np.mean(test_scores), color='blue', linestyle='--', alpha=0.5)
            axes[idx].axhline(y=np.mean(train_scores), color='red', linestyle='--', alpha=0.5)

            axes[idx].set_xlabel('Fold')
            axes[idx].set_ylabel(metric.upper())
            axes[idx].set_title(f'{metric.upper()} Across CV Folds')
            axes[idx].legend()
            axes[idx].grid(True, alpha=0.3)
            axes[idx].set_xticks(x_pos)

        plt.tight_layout()
        plt.show()

        # Model stability assessment
        f1_std = cv_stats['f1']['test_std']
        stability_assessment = "Stable" if f1_std < 0.05 else "Moderate" if f1_std < 0.10 else "Unstable"

        print(f"\n🎯 Model Stability Assessment:")
        print(f"• F1-Score Std Dev: {f1_std:.4f}")
        print(f"• Stability Rating: {stability_assessment}")
        print(f"• Max Overfitting: {max(cv_stats[m]['overfit'] for m in scoring):.4f} (F1)")

        if f1_std > 0.05:
            print("⚠️ Consider additional regularization or feature selection")
        else:
            print("✅ Model shows good stability across different data splits")

    except Exception as e:
        print(f"❌ Cross-validation failed: {e}")

else:
    print("⚠️ No models available for cross-validation")

In [ ]:
# Threshold Optimization for Best Model
if len(results_df) > 0:
    print("=== Threshold Optimization ===")

    # Get best model
    best_model_name = results_df.loc[results_df['F1'].idxmax()]['Model']
    best_pipeline = trained_models[best_model_name]

    # Get probability predictions
    y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

    # Test different thresholds (include 0.5 explicitly)
    thresholds = np.unique(np.concatenate([
        np.arange(0.1, 0.9, 0.05),
        [0.5]  # Ensure 0.5 is included
    ]))
    thresholds = np.sort(thresholds)
    threshold_results = []

    for threshold in thresholds:
        y_pred_thresh = (y_pred_proba >= threshold).astype(int)

        precision = precision_score(y_test, y_pred_thresh)
        recall = recall_score(y_test, y_pred_thresh)
        f1 = f1_score(y_test, y_pred_thresh)

        # Calculate business metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresh).ravel()
        investigation_load = tp + fp  # Total cases to investigate
        fraud_detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0

        threshold_results.append({
            'Threshold': threshold,
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
            'Investigation_Load': investigation_load,
            'Fraud_Detection_Rate': fraud_detection_rate,
            'True_Positives': tp,
            'False_Positives': fp
        })

    threshold_df = pd.DataFrame(threshold_results)

    # Find optimal thresholds for different objectives
    best_f1_idx = threshold_df['F1'].idxmax()
    best_precision_idx = threshold_df['Precision'].idxmax()

    # Business efficiency: maximize fraud detection while limiting investigations
    threshold_df['Efficiency'] = threshold_df['Fraud_Detection_Rate'] / (threshold_df['Investigation_Load'] / len(y_test))
    best_efficiency_idx = threshold_df['Efficiency'].idxmax()

    print(f"\n📊 Threshold Optimization Results for {best_model_name}:")
    print("=" * 80)
    print(f"{'Objective':<20} {'Threshold':<12} {'Precision':<12} {'Recall':<10} {'F1':<10} {'Investigations':<15}")
    print("-" * 80)
    print(f"{'Best F1':<20} {threshold_df.iloc[best_f1_idx]['Threshold']:<12.2f} {threshold_df.iloc[best_f1_idx]['Precision']:<12.3f} {threshold_df.iloc[best_f1_idx]['Recall']:<10.3f} {threshold_df.iloc[best_f1_idx]['F1']:<10.3f} {threshold_df.iloc[best_f1_idx]['Investigation_Load']:<15.0f}")
    print(f"{'Best Precision':<20} {threshold_df.iloc[best_precision_idx]['Threshold']:<12.2f} {threshold_df.iloc[best_precision_idx]['Precision']:<12.3f} {threshold_df.iloc[best_precision_idx]['Recall']:<10.3f} {threshold_df.iloc[best_precision_idx]['F1']:<10.3f} {threshold_df.iloc[best_precision_idx]['Investigation_Load']:<15.0f}")
    print(f"{'Best Efficiency':<20} {threshold_df.iloc[best_efficiency_idx]['Threshold']:<12.2f} {threshold_df.iloc[best_efficiency_idx]['Precision']:<12.3f} {threshold_df.iloc[best_efficiency_idx]['Recall']:<10.3f} {threshold_df.iloc[best_efficiency_idx]['F1']:<10.3f} {threshold_df.iloc[best_efficiency_idx]['Investigation_Load']:<15.0f}")

    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Metrics vs Threshold
    axes[0].plot(threshold_df['Threshold'], threshold_df['Precision'], 'b-', label='Precision', linewidth=2)
    axes[0].plot(threshold_df['Threshold'], threshold_df['Recall'], 'r-', label='Recall', linewidth=2)
    axes[0].plot(threshold_df['Threshold'], threshold_df['F1'], 'g-', label='F1-Score', linewidth=2)

    # Mark optimal points
    axes[0].scatter(threshold_df.iloc[best_f1_idx]['Threshold'], threshold_df.iloc[best_f1_idx]['F1'],
                   color='green', s=100, zorder=5, label='Optimal F1')

    axes[0].set_xlabel('Threshold')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Performance Metrics vs Threshold', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Investigation Load vs Fraud Detection Rate
    axes[1].scatter(threshold_df['Investigation_Load'], threshold_df['Fraud_Detection_Rate'],
                   c=threshold_df['F1'], cmap='viridis', s=60, alpha=0.7)

    # Mark efficient point
    axes[1].scatter(threshold_df.iloc[best_efficiency_idx]['Investigation_Load'],
                   threshold_df.iloc[best_efficiency_idx]['Fraud_Detection_Rate'],
                   color='red', s=100, zorder=5, label='Most Efficient')

    axes[1].set_xlabel('Investigation Load (Cases)')
    axes[1].set_ylabel('Fraud Detection Rate')
    axes[1].set_title('Investigation Efficiency Trade-off', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # Add colorbar
    cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
    cbar.set_label('F1-Score')

    plt.tight_layout()
    plt.show()

    # Recommendations
    print(f"\n💡 Threshold Recommendations:")
    print(f"• For balanced performance: {threshold_df.iloc[best_f1_idx]['Threshold']:.2f} (F1-optimized)")
    print(f"• For high confidence alerts: {threshold_df.iloc[best_precision_idx]['Threshold']:.2f} (Precision-optimized)")
    print(f"• For resource efficiency: {threshold_df.iloc[best_efficiency_idx]['Threshold']:.2f} (Investigation-optimized)")

    # Safely check for default threshold (0.5)
    try:
        # Try to find exact match first
        default_threshold_mask = (threshold_df['Threshold'] == 0.5)
        if default_threshold_mask.any():
            print(f"• Default threshold (0.5) F1-Score: {threshold_df[default_threshold_mask]['F1'].iloc[0]:.3f}")
        else:
            # Find the closest threshold to 0.5
            closest_idx = np.argmin(np.abs(threshold_df['Threshold'] - 0.5))
            closest_threshold = threshold_df.iloc[closest_idx]['Threshold']
            closest_f1 = threshold_df.iloc[closest_idx]['F1']
            print(f"• Closest to default threshold ({closest_threshold:.2f}) F1-Score: {closest_f1:.3f}")
    except (IndexError, KeyError) as e:
        print(f"• Default threshold comparison: Unable to compute (threshold range: {threshold_df['Threshold'].min():.2f}-{threshold_df['Threshold'].max():.2f})")

else:
    print("⚠️ No models available for threshold optimization")

In [ ]:
# Final recommendations and business impact analysis
if len(results_df) > 0:
    print("="*80)
    print("HEALTHCARE FRAUD DETECTION - COMPREHENSIVE ANALYSIS SUMMARY")
    print("="*80)

    best_model = results_df.loc[results_df['F1'].idxmax()]

    print(f"\n🎯 RECOMMENDED MODEL: {best_model['Model']}")
    print(f"\n📊 PERFORMANCE METRICS:")
    print(f"  • F1-Score: {best_model['F1']:.4f}")
    print(f"  • Precision: {best_model['Precision']:.4f}")
    print(f"  • Recall: {best_model['Recall']:.4f}")
    print(f"  • PR-AUC: {best_model['PR_AUC']:.4f}")
    print(f"  • ROC-AUC: {best_model['ROC_AUC']:.4f}")

    fraud_detected = int(best_model['Recall'] * (y_test == 1).sum())
    total_fraud = (y_test == 1).sum()
    total_providers = len(y_test)

    print(f"\n📈 BUSINESS IMPACT ANALYSIS:")
    print(f"  • Total providers evaluated: {total_providers:,}")
    print(f"  • Actual fraud cases: {total_fraud}")
    print(f"  • Fraud cases detected: {fraud_detected} out of {total_fraud}")
    print(f"  • Detection rate: {best_model['Recall']:.1%}")
    print(f"  • Precision rate: {best_model['Precision']:.1%}")
    print(f"  • False positive rate: {((1-best_model['Precision']) * fraud_detected/best_model['Precision'] if best_model['Precision'] > 0 else 0):.0f} unnecessary investigations")


    # Calculate potential savings
    avg_fraud_loss = 50000  # Assume $50k average fraud loss per case
    investigation_cost = 5000  # Assume $5k per investigation

    fraud_prevented = fraud_detected * avg_fraud_loss
    investigation_costs = (fraud_detected / best_model['Precision']) * investigation_cost if best_model['Precision'] > 0 else 0
    net_savings = fraud_prevented - investigation_costs

    print(f"\n💰 ESTIMATED FINANCIAL IMPACT (Annual):")
    print(f"  • Fraud prevented: ${fraud_prevented:,.0f}")
    print(f"  • Investigation costs: ${investigation_costs:,.0f}")
    print(f"  • Net savings: ${net_savings:,.0f}")
    print(f"  • ROI: {(net_savings/investigation_costs*100) if investigation_costs > 0 else 0:.1f}%")

    print(f"\n🏆 CLASS IMBALANCE STRATEGY EFFECTIVENESS:")
    print(f"  • Approach: Class Weighting")
    ratio = int((y == 0).sum()) / int((y == 1).sum())
    print(f"  • Original class ratio: {ratio:.1f}:1")
    print(f"  • Effective handling: {'✅ Yes' if best_model['F1'] > 0.7 else '⚠️ Moderate' if best_model['F1'] > 0.5 else '❌ Poor'}")
    print(f"  • Maintains data integrity: ✅")
    print(f"  • Computationally efficient: ✅")

    print(f"\n🔧 PRODUCTION DEPLOYMENT RECOMMENDATIONS:")
    print(f"  1. Primary Model: {best_model['Model']} with class weighting")
    print(f"  2. Optimal Threshold: Use threshold optimization results")
    print(f"  3. Monitoring: Track precision/recall drift monthly")
    print(f"  4. Retraining: Quarterly with new fraud patterns")
    print(f"  5. Interpretability: Implement feature importance tracking")
    print(f"  6. Alerts: Set up performance degradation warnings")
    print(f"  7. Compliance: Maintain audit trail for regulatory review")

    print(f"\n📋 MODEL VALIDATION CHECKLIST:")
    validation_checks = {
        "Cross-validation performed": "✅",
        "Class imbalance addressed": "✅",
        "Multiple algorithms compared": "✅",
        "Feature importance analyzed": "✅",
        "Threshold optimization done": "✅",
        "Business metrics calculated": "✅",
        "Overfitting checked": "✅" if 'cv_stats' in locals() else "⚠️"
    }

    for check, status in validation_checks.items():
        print(f"  {status} {check}")

    # Save comprehensive results
    results_df.to_csv('../data/model_results.csv', index=False)

    # Create deployment summary
    deployment_summary = {
        'best_model': best_model['Model'],
        'f1_score': best_model['F1'],
        'precision': best_model['Precision'],
        'recall': best_model['Recall'],
        'recommended_threshold': 0.5,  # Update with optimal threshold if calculated
        'deployment_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
        'total_features': X.shape[1],
        'training_size': len(y_train),
        'test_size': len(y_test)
    }

    # Save deployment info
    import json
    with open('../data/deployment_summary.json', 'w') as f:
        json.dump(deployment_summary, f, indent=2)

    print(f"\n💾 ARTIFACTS SAVED:")
    print(f"  • Model comparison: ../data/model_results.csv")
    print(f"  • Deployment summary: ../data/deployment_summary.json")

    print(f"\n" + "="*80)
    print("🎉 ANALYSIS COMPLETE - MODEL READY FOR DEPLOYMENT")
    print("="*80)

else:
    print("⚠️ No models were successfully trained. Please check your data and try again.")